In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
%matplotlib
%matplotlib inline
import numpy
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import math
import random

Using matplotlib backend: MacOSX


In [2]:
import gillespy2
from gillespy2.basic_root_solver import BasicRootSolver
from gillespy2.basic_ssa_solver import BasicSSASolver
from gillespy2.basic_hybrid_solver import BasicHybridSolver

Unable to use Cython optimized SSA:
Error:No module named 'pyximport'


In [3]:
class MichaelisMenten(gillespy2.Model):
     def __init__(self, parameter_values=None):
            #initialize Model
            gillespy2.Model.__init__(self, name="Michaelis_Menten")
            
            #parameters
            rate1 = gillespy2.Parameter(name='rate1', expression= 0.000017)
            rate2 = gillespy2.Parameter(name='rate2', expression= 0.005)
            rate3 = gillespy2.Parameter(name='rate3', expression = 0.001)
            self.add_parameter([rate1,rate2,rate3])
            
            #Species
            A = gillespy2.Species(name='A', initial_value=100)
            B = gillespy2.Species(name='B', initial_value=100)
            C = gillespy2.Species(name='C', initial_value=100)
            D = gillespy2.Species(name='D', initial_value=100)
            #DET = gillespy2.Species(name='DET', initial_value=1) #Create a deterministic species
            #Deterministic species will be a regular species, but with "deterministic" flag
            DET = gillespy2.Species(name='DET', initial_value=0, deterministic=True)
            self.add_species([A, B, C, D, DET])
            
            #rate_rules
            rate_rule1 = gillespy2.RateRule(name='rate_rule1', expression="sin(t)", species=DET)
            self.add_rate_rule(rate_rule1)
            
            #reactions
            r1 = gillespy2.Reaction(name="r1",reactants={A:1,B:1}, products={C:1},
                   rate=rate1)
            
            r2 = gillespy2.Reaction(name="r2",reactants={C:1}, products={A:1,B:1},
                    rate=rate2)
            
            r3 = gillespy2.Reaction(name="r3",reactants={C:1}, products={B:1,D:1},
                    rate=rate3)
            r4 = gillespy2.Reaction(name="r4", reactants={C:1}, products={D:1}, propensity_function="rate2*A*DET/vol")
            #r4 uses a deterministic species result as an argument
            self.add_reaction([r1,r2,r3, r4])
            self.timespan(numpy.linspace(0,100,1001))

In [4]:
model = MichaelisMenten()
print(model.listOfRateRules)

Adding rate rule  <gillespy2.gillespy2.RateRule object at 0x10d4424a8>  to  Michaelis_Menten
OrderedDict([('DET', <gillespy2.gillespy2.RateRule object at 0x10d4424a8>)])


In [ ]:
results = model.run(solver=BasicHybridSolver(), show_labels=True)
#print(results)

In [ ]:
plt.plot(results['time'], results['DET'], label='DET')
plt.legend(loc='best')

In [ ]:
plt.plot(results['time'], results['A'], label='A')
plt.plot(results['time'], results['B'], label='B')
plt.plot(results['time'], results['C'], label='C')
plt.plot(results['time'], results['D'], label='D')
plt.plot(results['time'], results['DET'], label='DET')
plt.legend(loc='best')

In [ ]:
plt.plot(results['time'], results['A'], label='A')
plt.plot(results['time'], results['B'], label='B')
plt.plot(results['time'], results['C'], label='C')
plt.plot(results['time'], results['D'], label='D')
plt.plot(results['time'], results['DET'], label='DET')
plt.legend(loc='best')

In [ ]:
print(results)

In [ ]:
#results = model.run(solver=BasicRootSolver(), show_labels=True)

In [ ]:
# plt.plot(results['time'], results['A'], label='A')
# plt.plot(results['time'], results['B'], label='B')
# plt.plot(results['time'], results['C'], label='C')
# plt.plot(results['time'], results['D'], label='D')
# plt.plot(results['time'], results['DET'], label='DET')
# plt.legend(loc='best')

In [ ]:
# print(results)

In [ ]:
numpy.linspace(0,100,1001)

In [ ]:
results['DET']

In [ ]:
def pend(y, t, b, c):
    theta, omega = y
    dydt = [omega, -b*omega - c*numpy.sin(theta)]
    return dydt
b = 0.25
c = 5.0
y0 = [numpy.pi - 0.1, 0.0]
t = numpy.linspace(0, 10, 101)

from scipy.integrate import odeint
sol = odeint(pend, y0, t, args=(b, c))

import matplotlib.pyplot as plt
plt.plot(t, sol[:, 0], 'b', label='theta(t)')
plt.plot(t, sol[:, 1], 'g', label='omega(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
sol.shape